In [1]:
import pandas as pd
import requests
import os
import dotenv   
from bs4 import BeautifulSoup


Get an api key from : https://apilayer.com/marketplace/google_search-api 

Then set it to a variable named 'google' in your .env file 

In [2]:
# import .env file
dotenv.load_dotenv()


True

In [3]:
# list of all 50 US states
united_states = ['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California', 'Colorado', 'Connecticut', 'Delaware',
                    'Florida', 'Georgia', 'Hawaii', 'Idaho', 'Illinois', 'Indiana', 'Iowa', 'Kansas', 'Kentucky',
                    'Louisiana', 'Maine', 'Maryland', 'Massachusetts', 'Michigan', 'Minnesota', 'Mississippi',
                    'Missouri', 'Montana', 'Nebraska', 'Nevada', 'New Hampshire', 'New Jersey', 'New Mexico',
                    'New York', 'North Carolina', 'North Dakota', 'Ohio', 'Oklahoma', 'Oregon', 'Pennsylvania',
                    'Rhode Island', 'South Carolina', 'South Dakota', 'Tennessee', 'Texas', 'Utah', 'Vermont',
                    'Virginia', 'Washington', 'West Virginia', 'Wisconsin', 'Wyoming']

In [4]:
# for state in united_states:
#   q = f'{state}%20storm%20article'
#   url = f"https://api.apilayer.com/google_search?q={q}"

#   payload = {}
#   headers= {
#   "apikey": os.getenv('google'),
#   }

#   response = requests.request("GET", url, headers=headers, data = payload)

#   status_code = response.status_code
#   result = response.text
#   with open(f'more_requests/{state}.json', 'w') as f:
#         f.write(response.text)
#   print(f'{state} status code: {status_code}')

In [5]:
# for state in united_states:
#   for i in range(10):

#       q = f'{state}%20storm%20article'
#       url = f"https://api.apilayer.com/google_search?q={q}&start={11+i}"

#       payload = {}
#       headers= {
#       "apikey": os.getenv('google'),
#       }

#       response = requests.request("GET", url, headers=headers, data = payload)

#       status_code = response.status_code
#       result = response.text
#       name = f'{state}_{i}'
#       with open(f'more_requests/{name}.json', 'w') as f:
#             f.write(response.text)
#       print(f'{state}_{i} status code: {status_code}')

In [6]:
import json
# urls
urls = {}

# step through the json files and get the urls for the articles
for file in os.listdir('./requests/'):
    with open(f'./requests/{file}') as f:
        urls[file[:-5]] = []
        data = json.load(f)
        for item in data['organic']:
            url = item['link']
            urls[file[:-5]].append(url)
urls

            

{'West Virginia': ['https://www.wowktv.com/top-stories/rounds-of-storms-for-west-virginia-kentucky-and-ohio/',
  'https://wvmetronews.com/2023/03/24/storm-headed-into-west-virginia-packs-a-punch/',
  'https://globalnews.ca/tag/west-virginia-storm/',
  'https://www.usatoday.com/story/news/nation/2023/02/17/west-virginia-weather-flooding-state-emergency/11279958002/',
  'https://www.weather.gov/rlx/snow',
  'https://wvpublic.org/strong-winds-storms-expected-across-the-state-saturday/',
  'https://www.wvgazettemail.com/news/wvs-eastern-mountains-brace-for-rare-may-snowstorm/article_0673c102-e6ef-5eb4-9454-b3882569a86e.html',
  'https://www.wboy.com/weather/say-it-aint-snow-west-virginia-mountains-to-see-few-inches-of-snow-in-early-may/',
  'https://stormhighway.com/wvstorms.php'],
 'Texas': ['https://www.texastribune.org/series/winter-storm-power-outage/',
  'https://www.nytimes.com/2023/01/31/us/ice-storm-winter-weather-texas.html',
  'https://www.cbsnews.com/news/texas-storms-heavy-rain

In [7]:
# add more reqeusts to the urls dictionary
for file in os.listdir('./more_requests/'):
    with open(f'./more_requests/{file}') as f:
        # urls[file[:-5]] = []
        data = json.load(f)
        for item in data['organic']:
            url = item['link']
            urls[file[:-7]].append(url)

In [8]:
# print the total number of urls
total = 0
for key in urls.keys():
    total += len(urls[key])
print(total)

5199


In [9]:
# download the articles from the urls in parallel
# articles = os.lisdir('./articles')

# for state in urls.keys():
#     print(state)
#     for i, url in enumerate(urls[state]):
#         print(i)
#         if f'{state}_{i}.html' in articles:
#             continue
#         try:
#             r = requests.get(url)
            
#             with open(f'./articles/{state}_{i}.html', 'w') as f:
#                 f.write(r.text)
#         except:
#             print(f'error with {state}_{i}')


In [10]:
# download the articles from the urls in parallel
articles = os.listdir('./articles')

def download_article(state, i, url):
    if f'{state}_{i}.html' in articles:
        return
    try:
        r = requests.get(url)
        
        with open(f'./articles/{state}_{i}.html', 'w') as f:
            f.write(r.text)
    except:
        print(f'error with {state}_{i}')

from joblib import Parallel, delayed
import multiprocessing

num_cores = multiprocessing.cpu_count()
print(num_cores)

Parallel(n_jobs=num_cores)(delayed(download_article)(state, i, url) for state in urls.keys() for i, url in enumerate(urls[state]))



12
error with Texas_48
error with Texas_89
error with Arizona_8
error with Idaho_2
error with Georgia_7
error with Alaska_9
error with Alaska_19
error with Alaska_20
error with Alaska_25
error with Alaska_52
error with Alaska_53
error with Alaska_61
error with Alaska_70
error with Alaska_82
error with Alaska_83
error with Missouri_73
error with Iowa_10
error with Iowa_21
error with Iowa_40
error with Iowa_54
error with Iowa_61
error with Iowa_88
error with Iowa_100
error with Alaska_89


KeyboardInterrupt: 

In [ ]:
# organize the articles into a dataframe
articles = os.listdir('./articles')

# create a dataframe
columns = ['title', 'author', 'publication', 'body_text' , 'url', 'state']
df = pd.DataFrame(columns=columns)

# step through the articles and add them to the dataframe
for article in articles:
    with open(f'./articles/{article}') as f:
        soup = BeautifulSoup(f.read(), 'html.parser')
        title = soup.find('title').text
        author = soup.find('meta', attrs={'name': 'author'})
        if author:
            author = author['content']
        publication = soup.find('meta', attrs={'name': 'publication'})
        if publication:
            publication = publication['content']
        body_text = soup.find('body').text
        url = soup.find('meta', attrs={'name': 'url'})
        if url:
            url = url['content']
        state = article.split('_')[0]
        df = df.append({'title': title, 'author': author, 'publication': publication, 'body_text': body_text, 'url': url, 'state': state}, ignore_index=True)